# Inductance simulation in COMSOL

In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["PMIX_MCA_gds"]="hash"

# Import useful packages
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from SQDMetal.Comps.Xmon import Xmon
from SQDMetal.Comps.Polygons import PolyRectangle
from SQDMetal.Comps.Inductors import InductorMeander
from SQDMetal.Comps.Joints import JointExtend
from SQDMetal.Comps.Wires import WirePins

# Set up chip design as planar
design = designs.DesignPlanar({}, overwrite_enabled=True)

# Set up chip dimensions 
design.chips.main.size.size_x = '250um'
design.chips.main.size.size_y = '250um'
design.chips.main.size.size_z = '250um'
design.chips.main.size.center_x = '0mm'
design.chips.main.size.center_y = '0mm'


InductorMeander(design, 'leInd', options=dict(pos_x='-50um', pos_y='0um', end_x='50um' , end_y='0um',
                                              track_width='2um', meander_spacing='4um', meander_width='30um', num_long_sections=10,
                                              init_gap='5um', mean_gap='5um'))
JointExtend(design, 'jntA', options=dict(jointObj='leInd', jointPin='a', dist_extend='20um', extend_off_pin_dir=True))
JointExtend(design, 'jntB', options=dict(jointObj='leInd', jointPin='b', dist_extend='20um', extend_off_pin_dir=True))
JointExtend(design, 'jntAa', options=dict(jointObj='jntA', jointPin='a', dist_extend='100um', orientation=90))
JointExtend(design, 'jntBa', options=dict(jointObj='jntB', jointPin='a', dist_extend='100um', orientation=90))
WirePins(design, 'wireFeed', options=Dict(pathObjPins=[('leInd', 'a'), 'jntA', 'jntAa', 'jntBa', 'jntB', ('leInd', 'b')], layer=7))

# gui = MetalGUI(design)
# gui.rebuild()
design.rebuild()

In [ ]:
from SQDMetal.COMSOL.Model import COMSOL_Model
from SQDMetal.COMSOL.SimInductance import COMSOL_Simulation_Inductance

COMSOL_Model.init_engine()
cmsl = COMSOL_Model('leModel')
sim_Inds = COMSOL_Simulation_Inductance(cmsl)

cmsl.initialize_model(design, [sim_Inds], bottom_grounded=True)

cmsl.add_metallic(1, threshold=1e-10, fuse_threshold=1e-10)
# cmsl.add_ground_plane()
cmsl.fuse_all_metals()

sim_Inds.set_return_loop_on_route('wireFeed')

# cmsl.set_current_feed_on_CPW_on_Route('wireFeed', pin_name='end', width_U = 20e-6, src_gnd_gap=10e-6)

# #(F) - OPTIONAL: Set everything within a rectangle to be finely meshed
# cmsl.fine_mesh_in_rectangle(-3e-3, -140e-6, -2.9e-3, 167e-6, 2e-7, 5e-6)
# #(G) - OPTIONAL: Set conductors within a rectangle to be finely meshed
# cmsl.fine_mesh_conductors_in_rectangle(-2.86e-3, -145e-6, -2.53e-3, 167e-6, 2e-7, 5e-6)

cmsl.build_geom_mater_elec_mesh(skip_meshing=True, mesh_structure='Fine')

# cmsl.save('Test')
inductance, resistance = sim_Inds.run()

In [56]:
inductance

3.659237343548751e-10